[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/introduction/19_Context_Module.ipynb)

# Deep Dive: Semantica Context Module (Architecture 2.0)

## Overview

The **Context Module** is the core state management system of Semantica. It allows agents to maintain coherent, persistent, and structured memory across long interactions. Unlike simple RAG systems that only use vector similarity, Semantica's Context Module combines:

1.  **FastEmbed Integration**: High-performance, local embedding generation.
2.  **Context Graph**: A structured knowledge graph for reasoning about relationships.
3.  **Hierarchical Memory**: A tiered system with short-term (token-limited) and long-term (vector-backed) storage.
4.  **Hybrid Retrieval**: Combining vector search, graph traversal (GraphRAG), and keyword matching.
5.  **Persistence**: Full state serialization.

This notebook provides a technical deep dive into these components.

**Documentation**: [API Reference](https://semantica.readthedocs.io/reference/context/)

---

## 1. Setup and Configuration

We need `semantica` and `fastembed` for local, high-speed embedding generation.

In [ ]:
!pip install semantica fastembed

In [ ]:
import os
import shutil
import numpy as np

from semantica.context import AgentContext, ContextGraph, AgentMemory, EntityLinker
from semantica.vector_store import VectorStore

## 2. Vector Store with FastEmbed

The `VectorStore` manages long-term memory. We will configure it to use **FastEmbed**, which runs efficient, quantized embedding models locally on the CPU.

We use the `inmemory` backend for this demo, but Semantica supports Qdrant, Weaviate, and FAISS for production.

In [ ]:
# 1. Initialize Vector Store
vs = VectorStore(backend="inmemory", dimension=384)

# 2. Configure FastEmbed
# We explicitly set the method to 'fastembed' and choose a lightweight, high-performance model.
if hasattr(vs, "embedder") and vs.embedder:
    print("Configuring VectorStore to use FastEmbed...")
    vs.embedder.set_text_model(
        method="fastembed", 
        model_name="BAAI/bge-small-en-v1.5"
    )

# 3. Verify Embedding Generation
text = "Semantica enables complex agent behaviors."
vector = vs.embed(text)
print(f"Generated embedding shape: {vector.shape}")

## 3. Context Graph Construction

The `ContextGraph` stores structured data. While vectors capture *similarity*, graphs capture *relationships*.

We will manually build a small graph to understand the API:
- `add_node(node_id, node_type, content, **properties)`
- `add_edge(source_id, target_id, edge_type, **properties)`

In [ ]:
kg = ContextGraph()

# Add Nodes
# Note: 'content' is what is indexed for keyword search.
kg.add_node(
    node_id="user_alice", 
    node_type="Person", 
    content="Alice", 
    role="Lead Engineer"
)
kg.add_node(
    node_id="tech_python", 
    node_type="Technology", 
    content="Python", 
    version="3.11"
)
kg.add_node(
    node_id="project_semantica", 
    node_type="Project", 
    content="Semantica Framework"
)

# Add Edges (Relationships)
kg.add_edge(source_id="user_alice", target_id="tech_python", edge_type="USES")
kg.add_edge(source_id="tech_python", target_id="project_semantica", edge_type="POWERS")

# Traverse the Graph
print("Neighbors of Python:")
neighbors = kg.get_neighbors("tech_python")
for n in neighbors:
    # The neighbor dict contains the connected node info and the relationship that led to it
    print(f" - [Relationship: {n['relationship']}] -> {n['content']} ({n['type']})")

## 4. AgentContext: The Unified Interface

`AgentContext` combines the `VectorStore` and `ContextGraph` into a single system. It handles:
1.  **Memory Management**: Routing inputs to short-term or long-term memory.
2.  **Hybrid Retrieval**: Querying both vectors and the graph simultaneously.

We will initialize it with limits to demonstrate the hierarchy.

In [ ]:
context = AgentContext(
    vector_store=vs,
    knowledge_graph=kg,
    token_limit=500,       # Max tokens in Short-Term Memory (STM)
    short_term_limit=5     # Max items in STM
)

# Store a new memory
# This is automatically embedded (FastEmbed) and indexed.
context.store(
    content="Alice is optimizing the graph traversal algorithms in Semantica.",
    conversation_id="dev_sync_1",
    user_id="alice"
)

# Retrieve Context
# 'use_graph=True' enables GraphRAG: it finds entities in the query ('Alice') 
# and expands to their neighbors in the graph.
results = context.retrieve(
    query="What is Alice working on?",
    use_graph=True,
    expand_graph=True
)

print("\n--- Hybrid Retrieval Results ---")
for res in results:
    print(f"[{res['score']:.2f}] {res['content']}")

## 5. Hierarchical Memory Management

Watch how the `AgentContext` manages memory pressure. We defined `short_term_limit=5`.
As we add more items, the oldest ones are flushed from the active buffer but remain safe in the Vector Store.

In [ ]:
print(f"STM Count (Start): {len(context.memory.short_term_memory)}")

# Fill up memory
for i in range(1, 10):
    context.store(f"Log entry {i}: System status check.")

print(f"STM Count (End): {len(context.memory.short_term_memory)}")
print("\nCurrent Short-Term Memory Items:")
for item in context.memory.short_term_memory:
    print(f" - {item.content}")

# Notice that earlier log entries are gone from this list, 
# but they are still retrievable via search.

## 6. Persistence

To build stateful agents, we must save and load the context. 
**Crucial Step**: When loading, we must ensure the new `VectorStore` instance is configured with the same embedding model (`FastEmbed`) so vectors match.

In [ ]:
SAVE_PATH = "./semantica_context_state"

# 1. Save state
print(f"Saving context to {SAVE_PATH}...")
context.save(SAVE_PATH)

# 2. Initialize a fresh AgentContext
print("Initializing fresh agent...")
new_kg = ContextGraph()
new_vs = VectorStore(backend="inmemory", dimension=384)

# !!! IMPORTANT: Re-configure FastEmbed before loading !!!
if hasattr(new_vs, "embedder") and new_vs.embedder:
    new_vs.embedder.set_text_model(
        method="fastembed", 
        model_name="BAAI/bge-small-en-v1.5"
    )

restored_context = AgentContext(vector_store=new_vs, knowledge_graph=new_kg)

# 3. Load state
restored_context.load(SAVE_PATH)

# 4. Verify restoration
print(f"Restored STM items: {len(restored_context.memory.short_term_memory)}")
print(f"Restored Graph nodes: {len(new_kg.nodes)}")

# Cleanup
if os.path.exists(SAVE_PATH):
    shutil.rmtree(SAVE_PATH)
    print("Cleanup complete.")

## Summary

You have successfully built a persistent, graph-aware, memory-managed context system using Semantica 2.0 components.

**Key Takeaways:**
- Use `VectorStore` with `FastEmbed` for efficient local vectors.
- Use `ContextGraph` to map relationships (`add_node`, `add_edge`).
- Use `AgentContext` to manage the lifecycle of memories and retrieval.
- Always configure your embedder on the fresh instance before calling `load()`.